In [41]:
using Random
using Distributions
using CUDA
using BenchmarkTools
using BinomialGPU

In [2]:
NUM_SPINS = 26
NUM_STATES = 10
OMP_THREADS = 4

4

In [59]:
graph_rand = rand(Uniform(-5,5),NUM_SPINS, NUM_SPINS)

26×26 Matrix{Float64}:
 -2.72238     4.69522     -1.46741   …   1.1384    -2.81682   -3.61539
 -4.03119     0.0482229   -1.41994       3.64753    3.91935    4.42098
  2.46478    -4.25474      0.614348      3.47913    4.32819    1.98736
 -2.28121     1.17585      1.09892      -4.54873    3.48605    4.5098
 -3.98394    -3.57021      0.78932      -2.74622   -2.29096   -3.10407
 -0.830255   -0.892752     0.757896  …  -1.55618    0.405363  -0.279909
  0.0157816  -2.47958     -0.902337     -0.608126  -4.27422    1.84045
 -4.33149    -2.82381      1.86028       1.78998   -1.57577    3.31655
  2.78204     1.07209     -4.35964       1.41214    1.31462    0.907664
 -0.230495   -0.401193     2.19253       1.96024   -2.6724    -1.19597
 -0.802828    2.52374     -1.53606   …   2.27009    0.763966  -2.4742
 -1.48556     3.05474      4.71179       4.83306   -4.47457    3.20602
  4.46694    -0.377336     2.88187       0.367546  -3.86627   -0.578989
  4.54358     3.44772     -4.00524       3.17724    0

In [55]:
@benchmark cu(rand(Uniform(-5,5),NUM_SPINS, NUM_SPINS))

BenchmarkTools.Trial: 
  memory estimate:  8.42 KiB
  allocs estimate:  10
  --------------
  minimum time:     4.015 μs (0.00% GC)
  median time:      4.587 μs (0.00% GC)
  mean time:        7.372 μs (7.32% GC)
  maximum time:     903.651 μs (55.58% GC)
  --------------
  samples:          10000
  evals/sample:     7

In [96]:
graph = Dict()
for i in 0:NUM_SPINS
    graph[(i, i)] = rand(Uniform(-5,5))
    for j in i:NUM_SPINS
        graph[(i, j)] = rand(Uniform(-5,5))
    end
end

In [97]:
graph

Dict{Any, Any} with 378 entries:
  (11, 17) => 1.75922
  (8, 15)  => 1.09126
  (16, 16) => -1.70168
  (7, 18)  => 4.71631
  (7, 8)   => 1.28611
  (14, 15) => -2.54694
  (7, 24)  => -1.54525
  (1, 9)   => -2.54711
  (15, 23) => -0.345505
  (2, 26)  => -3.36633
  (2, 17)  => 4.81567
  (10, 17) => -2.46452
  (10, 26) => 3.22568
  (18, 18) => 1.44323
  (18, 24) => 3.50805
  (20, 26) => -3.95342
  (17, 19) => -2.40701
  (16, 18) => -1.93971
  (16, 24) => 3.20406
  (6, 23)  => 0.398331
  (0, 26)  => 3.53226
  (0, 17)  => 3.58651
  (12, 21) => -2.49961
  (9, 26)  => -1.80702
  (9, 17)  => 1.51836
  ⋮        => ⋮

In [98]:
qubo = Dict()
constant = 0

for k in keys(graph)
   qubo[k] = 0 
end

for i in 0:NUM_SPINS
    qubo[(i,i)] = 2 * graph[(i,i)]
    constant += graph[(i,i)]
    
    for j in 0:NUM_SPINS
        if i!=j
            low, high = sort!([i, j])
            constant -= graph[(low,high)]*0.5
            qubo[(i,i)] -= 2*graph[(low,high)]
            qubo[(low,high)] += graph[(low,high)]*2
        end
    end
end


In [99]:
qubo

Dict{Any, Any} with 378 entries:
  (11, 17) => 7.03688
  (8, 15)  => 4.36503
  (16, 16) => 2.68409
  (7, 18)  => 18.8652
  (7, 8)   => 5.14446
  (14, 15) => -10.1878
  (7, 24)  => -6.18101
  (1, 9)   => -10.1884
  (15, 23) => -1.38202
  (2, 26)  => -13.4653
  (2, 17)  => 19.2627
  (10, 17) => -9.85807
  (10, 26) => 12.9027
  (18, 18) => -21.5527
  (18, 24) => 14.0322
  (20, 26) => -15.8137
  (17, 19) => -9.62805
  (16, 18) => -7.75883
  (16, 24) => 12.8162
  (6, 23)  => 1.59332
  (0, 26)  => 14.129
  (0, 17)  => 14.3461
  (12, 21) => -9.99846
  (9, 26)  => -7.22808
  (9, 17)  => 6.07345
  ⋮        => ⋮

In [171]:
@btime begin

    NUM_SPINS = 20

    graph = Dict()
    for i in 0:NUM_SPINS
        graph[(i, i)] = rand(Uniform(-5,5))
        for j in i:NUM_SPINS
            graph[(i, j)] = rand(Uniform(-5,5))
        end
    end

    qubo = Dict()
    constant = 0

    for k in keys(graph)
       qubo[k] = 0 
    end

    for i in 0:NUM_SPINS
        qubo[(i,i)] = 2 * graph[(i,i)]
        constant += graph[(i,i)]

        for j in 0:NUM_SPINS
            if i!=j
                low, high = sort!([i, j])
                constant -= graph[(low,high)]*0.5
                qubo[(i,i)] -= 2*graph[(low,high)]
                qubo[(low,high)] += graph[(low,high)]*2
            end
        end
    end

    res = Vector{Float64}()
#     res = Dict()
    
    N = NUM_SPINS
    for (k, q) in enumerate(Iterators.product(fill(0:1,N)...))
        F = 0
        for i in 1:NUM_SPINS
            F -= qubo[(i-1,i-1)]*q[i]  
            for j in 1:NUM_SPINS
                low, high = sort!([i-1, j-1])
                F -= qubo[(low,high)]*q[i]*q[j]
            end
        end
        append!(res, F) 
#         res[k] = F
    end

end
res

  348.564 s (1406147982 allocations: 56.38 GiB)


32-element Vector{Float64}:
   0.0
 -44.084180314968364
 -14.012573750590327
 -58.32324860101298
 -34.61669627695693
 -77.75803491438023
  -9.88066960713104
 -53.248502780008636
 -45.798559096767235
 -72.27180397061437
 -25.78325623255217
 -52.48299564185359
 -85.34097945798413
   ⋮
 -78.49211508105091
 -97.81487089009175
 -67.56889096267716
 -87.1181413071723
 -34.89711167352614
 -37.55177371899082
 -28.69461136076321
 -31.575767941682187
 -89.76581450187496
 -91.47763486979457
 -44.8147137686958
 -46.75302867206973

In [167]:
resd = Dict(zip(1:size(res)[1],res))

sort(collect(resd), by=x->x[2])

32-element Vector{Pair{Int64, Float64}}:
 14 => -110.87138265428621
 22 => -97.81487089009175
 30 => -91.47763486979457
 29 => -89.76581450187496
 24 => -87.1181413071723
 13 => -85.34097945798413
 21 => -78.49211508105091
  6 => -77.75803491438023
 20 => -76.86660466104473
 10 => -72.27180397061437
 23 => -67.56889096267716
  4 => -58.32324860101298
 19 => -56.374512639004536
    ⋮
  2 => -44.084180314968364
 26 => -37.55177371899082
 25 => -34.89711167352614
  5 => -34.61669627695693
 28 => -31.575767941682187
 27 => -28.69461136076321
 17 => -28.54913633696207
 15 => -26.577076173352843
 11 => -25.78325623255217
  3 => -14.012573750590327
  7 => -9.88066960713104
  1 => 0.0

https://shinaoka.github.io/hpc_julia/docs/1dIsing.html

In [2]:
using BenchmarkTools, Random

In [1]:
function ising1d!(s, β, niters, rng)
    n = length(s)
    min_h = -2
    max_h = 2
    prob = [1/(1+exp(-2*β*h)) for h in min_h:max_h]
    #prob_f(h) = 1/(1+exp(-2*β*h))
    for iter in 1:niters, i in 1:n
        sl = s[ifelse(i == 1, n, i-1)]
        sr = s[ifelse(i == n, 1, i+1)]
        # h = -2, 0, 2
        h = sl + sr
        s[i] = ifelse(rand(rng) < prob[h-min_h+1], +1, -1)
        #s[i] = ifelse(rand(rng) < prob_f(h), +1, -1)
    end
end

num_spins = 100
rng = MersenneTwister(4649)
s0 = rand(rng, Int8[-1, 1], num_spins)
β = 10.0
niters = 10^3

s = copy(s0)

# Run once to compile the function
ising1d!(s, β, niters, rng)

@time ising1d!(s, β, niters, rng)
@benchmark ising1d!(s, β, niters, rng) setup=(s = copy(s0))

LoadError: UndefVarError: MersenneTwister not defined

In [6]:
def ising_to_qubo(ham):
    qubo = numpy.zeros(ham.shape)
    constant = 0
    for i in range(qubo.shape[0]):
        qubo[i, i] = 2 * ham[i, i]
        constant +=  ham[i, i]
        for j in range(qubo.shape[0]):
            if i != j:
                low, high = sorted((i, j))
                constant -= ham[low, high] * 0.5
                qubo[i, i] -= 2 * ham[low, high]
                qubo[low, high] += ham[low, high] * 2
    return qubo, constant


LoadError: TypeError: in typeassert, expected Vector{Float64}, got a value of type Vector{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}}

https://www.tensors.net/exact-diagonalization

In [16]:
using Pkg

In [17]:
Pkg.add("Arpack")

   Resolving package versions...
    Updating `~/Workspace/Repo/Code/SpinGlassExhaustive.jl/Project.toml`
  [7d9fca2a] + Arpack v0.4.0
  No Changes to `~/Workspace/Repo/Code/SpinGlassExhaustive.jl/Manifest.toml`


In [19]:
"""
doApplyHam(psiIn,hloc,N,usePBC)
------------------------
by Glen Evenbly (c) for www.tensors.net, (v1.2) - last modified 06/2020
------------------------
Applies local Hamiltonian (given as sum of nearest neighbor terms 'hloc')
to input state 'psiIn'. Number of lattice sites specified as 'N' while
'usePBC' determines whether open or periodic boundaries are used.
"""
function doApplyHam(psiIn,hloc,N,usePBC)

  d = size(hloc,1);
  psiOut = zeros(d^N,1);
  for k = 1:N-1
    # apply local Hamiltonian terms to sites [k,k+1]
    cont_inds = [[2],[2]];
    psi_temp = tensordot(reshape(hloc,d^2,d^2),
      reshape(psiIn,d^(k-1), d^2, d^(N-1-k)),cont_inds);
    psiOut = psiOut + reshape(permutedims(psi_temp,[2,1,3]),d^N);
  end

  if usePBC
    # apply periodic term
    cont_inds = [[3,4],[3,1]];
    psi_temp = tensordot(reshape(hloc,d,d,d,d),
      reshape(psiIn,d, d^(N-2), d),cont_inds);
    psiOut = psiOut + reshape(permutedims(psi_temp,[2,3,1]),d^N);
  end

  return psiOut
end


"""
tensordot(A,B,cont_inds):
Function for taking the produce of two tensors A and B, designed to mimic the
numpy tensordot. Input `cont_inds = [A_axes,B_axes]` where `A_axes` and
`B_axes` are vectors describing the indices to be contracted, e.g. set
cont_inds = [[2],[1]] to contract the 2nd index of A with the 1st index of B.
"""
function tensordot(A, B, cont_inds)

  A_free = deleteat!(collect(1:ndims(A)), sort(cont_inds[1]))
  B_free = deleteat!(collect(1:ndims(B)), sort(cont_inds[2]))
  A_perm = vcat(A_free, cont_inds[1])
  B_perm = vcat(cont_inds[2], B_free)

  return reshape(
    reshape(
      permutedims(A, A_perm),
      prod(size(A)[A_free]),
      prod(size(A)[cont_inds[1]]),
    ) * reshape(
      permutedims(B, B_perm),
      prod(size(B)[cont_inds[2]]),
      prod(size(B)[B_free]),
    ),
    (size(A)[A_free]..., size(B)[B_free]...),
  )
end

tensordot

In [20]:
"""
    ncon(tensor_list, connect_list_in; con_order=[], check_network=true)
------------------------
    by Glen Evenbly (c) for www.tensors.net, (v1.2) - last modified 6/2020
------------------------
Network CONtractor. Input is an array of tensors 'tensor_list' and an array of
vectors 'connect_list_in',  with each vector labelling the indices of the
corresponding tensor. Labels should be  positive integers for contracted indices
and negative integers for free indices. Optional input 'con_order'  can be used
to specify order of index contractions (otherwise defaults to ascending order of
the positive indices). Checking of the consistancy of the input network can be
disabled for slightly faster operation.

Further information can be found at: https://arxiv.org/abs/1402.0939
"""
function ncon(
  tensor_list,
  connect_list_in;
  con_order = [],
  check_network = true,
)

  # copy original list to avoid destroying
  connect_list = deepcopy(connect_list_in)

  # put inputs into an array if necessary
  if (tensor_list[1] isa Real) | (tensor_list[1] isa Complex)
    tensor_list = Any[tensor_list]
  end
  if !(connect_list[1] isa Array)
    connect_list = Any[connect_list]
  end

  # generate contraction order if necessary
  flat_connect = vcat(connect_list...)
  if isempty(con_order)
    con_order = sort(unique(flat_connect[flat_connect.>0]))
  end

  # check inputs if enabled
  if check_network
    dims_list = Array{Any,1}(undef, length(tensor_list))
    for ik = 1:length(tensor_list)
      dims_list[ik] = [size(tensor_list[ik])...]
    end
    ncon_check_inputs(connect_list, flat_connect, dims_list, con_order)
  end

  # do all partial traces
  for ip = 1:length(connect_list)
    num_cont = length(connect_list[ip]) - length(unique(connect_list[ip]))
    if num_cont > 0
      tensor_list[ip], connect_list[ip], cont_label =
        ncon_partial_trace(tensor_list[ip], connect_list[ip])
      con_order = setdiff(con_order, cont_label)
    end
  end

  # do all binary contractions
  while !isempty(con_order)
    # identify tensors to be contracted
    cont_ind = con_order[1];
    locs = [
      ele
      for
      ele in collect(1:length(connect_list)) if
      sum(connect_list[ele] .== cont_ind) > 0
    ]

    # do a binary contraction
    cont_many = intersect(connect_list[locs[1]], connect_list[locs[2]])
    A_cont = [findfirst(connect_list[locs[1]] .== x) for x in cont_many]
    B_cont = [findfirst(connect_list[locs[2]] .== x) for x in cont_many]
    A_free = deleteat!(collect(1:length(connect_list[locs[1]])), sort(A_cont))
    B_free = deleteat!(collect(1:length(connect_list[locs[2]])), sort(B_cont))
    push!(
      tensor_list,
      ncon_tensordot(
        tensor_list[locs[1]],
        tensor_list[locs[2]],
        A_cont,
        B_cont,
      ),
    )
    push!(
      connect_list,
      vcat(connect_list[locs[1]][A_free], connect_list[locs[2]][B_free]),
    )

    # remove contracted tensors from list and update con_order
    deleteat!(connect_list, locs)
    deleteat!(tensor_list, locs)
    con_order = setdiff(con_order, cont_many)
  end

  # do all outer products
  while length(tensor_list) > 1
    s1 = size(tensor_list[end-1])
    s2 = size(tensor_list[end])
    tensor_list[end-1] = reshape(
      reshape(tensor_list[end-1], prod(s1)) *
      reshape(tensor_list[end], 1, prod(s2)),
      (s1..., s2...),
    )
    connect_list[end-1] = vcat(connect_list[end-1], connect_list[end])
    deleteat!(connect_list, length(connect_list))
    deleteat!(tensor_list, length(tensor_list))
  end

  # do final permutation
  if length(connect_list[1]) > 0
    return permutedims(tensor_list[1], sortperm(connect_list[1], by = abs))
  else
    return tensor_list[1][1]
  end
end

"""
ncon_tensordot: contracts a pair of tensors via matrix multiplication,
similar to the Numpy function of the same name
"""
function ncon_tensordot(A, B, A_cont, B_cont)

  A_free = deleteat!(collect(1:ndims(A)), sort(A_cont))
  B_free = deleteat!(collect(1:ndims(B)), sort(B_cont))
  A_perm = vcat(A_free, A_cont)
  B_perm = vcat(B_cont, B_free)

  return reshape(
    reshape(
      permutedims(A, A_perm),
      prod(size(A)[A_free]),
      prod(size(A)[A_cont]),
    ) * reshape(
      permutedims(B, B_perm),
      prod(size(B)[B_cont]),
      prod(size(B)[B_free]),
    ),
    (size(A)[A_free]..., size(B)[B_free]...),
  )
end

"""
ncon_partial_trace: partial trace on tensor A over repeated labels in A_label
"""
function ncon_partial_trace(A, A_label)

  num_cont = length(A_label) - length(unique(A_label))
  if num_cont > 0
    dup_list = []
    for ele in unique(A_label)
      if sum(A_label .== ele) > 1
        dup_list = vcat(dup_list, findall(A_label .== ele))
      end
    end

    cont_ind =
      reshape(permutedims(reshape(dup_list, 2, num_cont), [2, 1]), 2 * num_cont)
    free_ind = deleteat!(collect(1:length(A_label)), sort(dup_list))
    cont_dim = prod(size(A)[cont_ind[1:num_cont]])
    free_dim = size(A)[free_ind]

    cont_label = unique(A_label[cont_ind])
    B = zeros(prod(free_dim))
    perm_tot = [free_ind; cont_ind]

    A_dims = size(A)
    A = reshape(
      permutedims(reshape(A[:], A_dims), vcat(free_ind, cont_ind)),
      prod(free_dim),
      cont_dim,
      cont_dim,
    )
    for ip = 1:cont_dim
      B = B + A[:, ip, ip]
    end

    return reshape(B, free_dim), deleteat!(A_label, sort(cont_ind)), cont_label
  else
    return A, A_label
  end
end

"""
ncon_check_inputs: check consistency of input tensor network
"""
function ncon_check_inputs(connect_list, flat_connect, dims_list, con_order)

  pos_ind = flat_connect[flat_connect.>0]
  neg_ind = flat_connect[flat_connect.<0]

  # check that lengths of lists match
  if length(dims_list) != length(connect_list)
    e_str0 = "NCON error: $(length(dims_list)) tensors given ";
    error(e_str0,"but $(length(connect_list)) index sublists given")
  end

  # check that tensors have the right number of indices
  for ik = 1:length(dims_list)
    if length(dims_list[ik]) != length(connect_list[ik])
      e_str0 = "number of indices does not match number of labels on tensor ";
      e_str1 = "$(ik): $(length(dims_list[ik]))-indices "
      error(e_str0,e_str1,"versus $(length(connect_list[ik]))-labels")
    end
  end

  # check that contraction order is valid
  if !(sort(con_order) == sort(unique(pos_ind)))
    error("NCON error: invalid contraction order")
  end

  # check that negative indices are valid
  for ind = -1:-1:-length(neg_ind)
    if sum(neg_ind .== ind) == 0
      error("NCON error: no index labelled $(ind)")
    elseif sum(neg_ind .== ind) > 1
      error("NCON error: more than one index labelled $(ind)")
    end
  end

  # check that positive indices are valid and contracted tensor dimensions match
  flat_dims = []
  for ele in dims_list
    flat_dims = vcat(flat_dims, ele)
  end
  for ind in unique(pos_ind)
    if sum(pos_ind .== ind) == 1
      error("NCON error: only one index labelled $(ind)")
    elseif sum(pos_ind .== ind) > 2
      error("NCON error: more than two indices labelled $(ind)")
    end
    cont_dims = flat_dims[flat_connect.==ind]
    if cont_dims[1] != cont_dims[2]
      e_str0 = "dimension mismatch on index labelled $(ind): "
      error(e_str0,"dim-$(cont_dims[1]) versus dim-$(cont_dims[2])")
    end
  end

  return true
end

ncon_check_inputs

In [22]:
"""
mainExactDiag.jl
---------------------------------------------------------------------
Script file for initializing exact diagonalization using the 'eigs' routine
(based on Arpack) for a 1D quantum system

by Glen Evenbly (c) for www.tensors.net, (v1.2) - last modified 06/2020
"""

using Printf, LinearAlgebra, LinearMaps, Arpack

#### Your paths go here
# include("ncon.jl");
# include("doApplyHam.jl");

##### Simulation parameters
model = "XX" # select 'XX' or 'ising' models
N = 18; # number of lattice sites
usePBC = true; # use periodic or open boundaries
numval = 1; # number of eigenstates to compute
Nsites = 18
##### Define Hamiltonian (quantum XX model)
d = 2; # local dimension
sX = [0 1; 1 0]; sY = [0 -im; im 0]; sZ = [1 0; 0 -1]; sI = [1 0; 0 1];
if model == "XX"
    hloc = reshape(real(kron(sX,sX) + kron(sY,sY)),2,2,2,2);
    EnExact = -4/sin(pi/Nsites); # for PBC
elseif model == "ising"
    hloc = reshape(-kron(sX,sX) + 0.5*kron(sI,sZ) + 0.5*kron(sZ,sI),2,2,2,2);
    EnExact = -2/sin(pi/(2*Nsites)); # for PBC
end

##### Recast the `doApplyHam` function as a LinearMap
doApplyHamClosed = LinearMap(psiIn -> doApplyHam(psiIn,hloc,N,usePBC), d^N;
  ismutating=false, issymmetric=true, ishermitian=true, isposdef=false)

##### Diagonalize Hamiltonian with eigs
diagtime = @elapsed Energy, psi = eigs(doApplyHamClosed; nev = numval,
  tol = 1e-12, which=:SR, maxiter = 300);

##### Check with exact energy
EnErr = Energy[1] - EnExact; # should equal to zero
@printf "NumSites: %d, Time: %d, Energy: %e, EnErr: %e \n" Nsites diagtime Energy[1] EnErr




NumSites: 18, Time: 8, Energy: -2.303508e+01, EnErr: 3.552714e-15 


In [23]:
CUDA.limit(CUDA.LIMIT_DEV_RUNTIME_SYNC_DEPTH)

LoadError: UndefVarError: CUDA not defined

In [24]:
using CUDA

function body(arr, my_shmem1, my_shmem2, i)
    my_shmem1[i] = arr[i] + 1
    my_shmem2[i] = arr[i]
    sync_threads()
    arr[i] = my_shmem1[i] - my_shmem2[i]
end

function static(arr :: CuDeviceArray{T}) where {T}
    i = threadIdx().x
    my_shmem1 = @cuStaticSharedMem(T, 1)
    my_shmem2 = @cuStaticSharedMem(T, 1)
    body(arr, my_shmem1, my_shmem2, i)
    return nothing
end

function dynamic(arr :: CuDeviceArray{T}) where {T}
    i = threadIdx().x
    my_shmem1 = @cuDynamicSharedMem(T, 1)
    my_shmem2 = @cuDynamicSharedMem(T, 1)
    body(arr, my_shmem1, my_shmem2, i)
    return nothing
end

N = 1
T = Int32
a = ones(T, 1)
b = CuArray(a)
c = CuArray(a)

@cuda threads=N static(b)
synchronize()

@cuda threads=N shmem=sizeof(T) * 2 * N dynamic(c)
synchronize()

@info "b = $b"
@info "c = $c"

┌ Info: b = Int32[1]
└ @ Main In[24]:38
┌ Info: c = Int32[0]
└ @ Main In[24]:39
